In [1]:
import tensorflow as tf
import time
import codecs
import os
import collections
from six.moves import cPickle
import numpy as np

In [2]:
class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding='utf-8'):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.encoding = encoding

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    def preprocess(self, input_file, vocab_file, tensor_file):
        with codecs.open(input_file, "r", encoding=self.encoding) as f:
            data = f.read()
        counter = collections.Counter(data)
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.chars, f)
        self.tensor = np.array(list(map(self.vocab.get, data)))
        np.save(tensor_file, self.tensor)

    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.chars = cPickle.load(f)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

        # When the data (tensor) is too small, let's give them a better error message
        if self.num_batches==0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)


    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

In [3]:
seq_length = 50 # RNN sequence length
batch_size = 60  # minibatch size, i.e. size of data in each epoch
num_epochs = 125 # you should change it to 50 if you want to see a relatively good results
learning_rate = 0.002
decay_rate = 0.97
rnn_size = 128 # size of RNN hidden state (output dimension)
num_layers = 2 #number of layers in the RNN

In [4]:
!wget -nv -O input.txt https://ibm.box.com/shared/static/a3f9e9mbpup09toq35ut7ke3l3lf03hg.txt 
with open('input.txt', 'r') as f:
    read_data = f.read()
    print read_data[0:100]
f.closed

2017-11-05 00:59:30 URL:https://public.boxcloud.com/d/1/-gEkL6nVNi4rShiV3so5-3stqsUY-sxj0YJIHGxx1VAl9RE2cV6WAVngkjTOGMFgRP_NywXxGwE6FyLudpBAz-Vje8TrdClkB5jC5eZlh-DqLWdwXlaojWg7kEk6KuTFfkYxYK8fSA4deDByZje16CNmuZR2zu_K2OaBKtId2C3MdlON4D8B4d0d0eM8Sl084Qx70uLizPtp0xnCQs80yVfag_gPIyOHNN-C_TUfjftXa8xRMZjj9bEMoMfQIdK84BOlqEGRqlXrtFvQUvH3qNLYsGwZaXNcIWGPvRrAH9YyLz0dBxxSmfoqeB4yD2GiJBHRCrQaK6a2vQVBAP6BGlWuBcDEjGluIWAds0t-7MvImgq0fvndgaSnMhVy8fZUOJnm9TDDqhBxKcN5EwKZ2uM64ODs5TbTZTY6gIS-UCLaxKascPLbLHpMYW039hytb5eRRcBNllYQHHhqKGj2BTyF8UiSJnk6GwhbzXqL5xE0Xq0nIGqNL_P7tViMn94_761z4So7ENTpF5rxBf1YfS9HqhTbU3givrXQAZGK0fij9Axmfp_bdx9_zdREANEIGG6E8siwdvfBXEAg04ygE-xkkAasrYSqBE03LUbtD5CY1GPKtl4gC48jTem1RBtME1KlVyJaRr1J8j3CpVtFWSr8DZ0UqNRiBzHLqtxcQ_1xlv-RfEWVBC_ksmAZ_3d5olnBz0IU1mxwP9Pp8KbgU1zoG1oLodtAYFBE9HJSejwMvlUMIT2ltgqoB3JB_Ug4aVNVulm7j27sQ4p7w1ejEhZj2C6s_qhuqIQ7T4TpiKnblkqP6mXIaqGv_sWiNkvmpikM3ez1WwZDcKj0g6GF4Zxk_a56cqt7cCctcmbW_kZ6Ug0dVT2ORQlwl_6dXFNWKCIdonuTHkKhIgaB4rwJvZ3CQYjNkcD46CEonfQ5z0ooxqf8

True

In [5]:
data_loader = TextLoader('', batch_size, seq_length)
vocab_size = data_loader.vocab_size
print "vocabulary size:" ,data_loader.vocab_size
print "Characters:" ,data_loader.chars
print "vocab number of 'F':",data_loader.vocab['F']
print "Character sequences (first batch):", data_loader.x_batches[0]

reading text file
vocabulary size: 65
Characters: (u' ', u'e', u't', u'o', u'a', u'h', u's', u'r', u'n', u'i', u'\n', u'l', u'd', u'u', u'm', u'y', u',', u'w', u'f', u'c', u'g', u'I', u'b', u'p', u':', u'.', u'A', u'v', u'k', u'T', u"'", u'E', u'O', u'N', u'R', u'S', u'L', u'C', u';', u'W', u'U', u'H', u'M', u'B', u'?', u'G', u'!', u'D', u'-', u'F', u'Y', u'P', u'K', u'V', u'j', u'q', u'x', u'z', u'J', u'Q', u'Z', u'X', u'3', u'&', u'$')
vocab number of 'F': 49
Character sequences (first batch): [[49  9  7 ...,  1  4  7]
 [19  4 14 ..., 14  9 20]
 [ 8 20 10 ...,  8 10 18]
 ..., 
 [21  2  0 ...,  0 21  0]
 [ 9  7  7 ...,  0  2  3]
 [ 3  7  0 ...,  5  9 23]]


In [6]:
x,y = data_loader.next_batch()
x

array([[49,  9,  7, ...,  1,  4,  7],
       [19,  4, 14, ..., 14,  9, 20],
       [ 8, 20, 10, ...,  8, 10, 18],
       ..., 
       [21,  2,  0, ...,  0, 21,  0],
       [ 9,  7,  7, ...,  0,  2,  3],
       [ 3,  7,  0, ...,  5,  9, 23]])

In [7]:
x.shape

(60, 50)

In [8]:
y

array([[ 9,  7,  6, ...,  4,  7,  0],
       [ 4, 14, 22, ...,  9, 20,  5],
       [20, 10, 29, ..., 10, 18,  4],
       ..., 
       [ 2,  0,  6, ..., 21,  0,  6],
       [ 7,  7,  4, ...,  2,  3,  0],
       [ 7,  0, 33, ...,  9, 23,  0]])

In [9]:
cell = tf.contrib.rnn.BasicRNNCell(rnn_size)

In [10]:
stacked_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)

In [11]:
stacked_cell.output_size

128

In [12]:
stacked_cell.state_size

(128, 128)

In [13]:
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])
input_data

<tf.Tensor 'Placeholder:0' shape=(60, 50) dtype=int32>

In [14]:
targets = tf.placeholder(tf.int32, [batch_size, seq_length])
targets

<tf.Tensor 'Placeholder_1:0' shape=(60, 50) dtype=int32>

In [15]:
initial_state = stacked_cell.zero_state(batch_size, tf.float32)

In [16]:
session = tf.Session()
feed_dict={input_data:x, targets:y}
session.run(input_data, feed_dict)

array([[49,  9,  7, ...,  1,  4,  7],
       [19,  4, 14, ..., 14,  9, 20],
       [ 8, 20, 10, ...,  8, 10, 18],
       ..., 
       [21,  2,  0, ...,  0, 21,  0],
       [ 9,  7,  7, ...,  0,  2,  3],
       [ 3,  7,  0, ...,  5,  9, 23]], dtype=int32)

In [17]:
with tf.variable_scope('rnnlm', reuse=False):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size]) #128x65
    softmax_b = tf.get_variable("softmax_b", [vocab_size]) # 1x65)
    #with tf.device("/cpu:0"):
        
    # embedding variable is initialized randomely
    embedding = tf.get_variable("embedding", [vocab_size, rnn_size])  #65x128

    # embedding_lookup goes to each row of input_data, and for each character in the row, finds the correspond vector in embedding
    # it creates a 60*50*[1*128] matrix
    # so, the first elemnt of em, is a matrix of 50x128, which each row of it is vector representing that character
    em = tf.nn.embedding_lookup(embedding, input_data) # em is 60x50x[1*128]
    # split: Splits a tensor into sub tensors.
    # syntax:  tf.split(split_dim, num_split, value, name='split')
    # it will split the 60x50x[1x128] matrix into 50 matrix of 60x[1*128]
    inputs = tf.split(em, seq_length, 1)
    # It will convert the list to 50 matrix of [60x128]
    inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

In [18]:
session.run(tf.global_variables_initializer())
session.run(embedding)

array([[ 0.1233625 , -0.14391455, -0.013116  , ..., -0.04574051,
        -0.11292589, -0.0732579 ],
       [ 0.01130158, -0.14711592, -0.1024131 , ...,  0.09682466,
        -0.05894173, -0.10901564],
       [-0.00731142, -0.02620618, -0.1413168 , ...,  0.17210825,
         0.03168036,  0.15227173],
       ..., 
       [-0.04203579, -0.11600257, -0.02687182, ...,  0.0466167 ,
         0.1256734 ,  0.00315903],
       [-0.08925935,  0.03854302, -0.11006494, ..., -0.11124674,
        -0.01411405, -0.03192475],
       [ 0.08436356,  0.13916643, -0.04529156, ...,  0.05557318,
         0.05172578,  0.09523539]], dtype=float32)

In [19]:
em = tf.nn.embedding_lookup(embedding, input_data)
emp = session.run(em,feed_dict={input_data:x})
print emp.shape
emp[0]

(60, 50, 128)


array([[ 0.10953732,  0.11556537,  0.14913632, ...,  0.07149842,
        -0.12136988, -0.02237552],
       [-0.03133665, -0.10970489, -0.14375091, ...,  0.07577346,
        -0.11832098,  0.10311176],
       [ 0.04275867,  0.16907538, -0.13553825, ...,  0.11105661,
         0.06070592,  0.00575978],
       ..., 
       [ 0.01130158, -0.14711592, -0.1024131 , ...,  0.09682466,
        -0.05894173, -0.10901564],
       [-0.11554965, -0.14424968, -0.12309102, ..., -0.17461821,
         0.11136095,  0.17032541],
       [ 0.04275867,  0.16907538, -0.13553825, ...,  0.11105661,
         0.06070592,  0.00575978]], dtype=float32)

In [20]:
inputs = tf.split(em, seq_length, 1)
inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
inputs[0:5]

[<tf.Tensor 'Squeeze:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_1:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_2:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_3:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_4:0' shape=(60, 128) dtype=float32>]

In [21]:
session.run(inputs[0],feed_dict={input_data:x})

array([[ 0.10953732,  0.11556537,  0.14913632, ...,  0.07149842,
        -0.12136988, -0.02237552],
       [ 0.1622739 , -0.05693389,  0.08251728, ...,  0.08289652,
         0.05217007,  0.07857351],
       [-0.11087403,  0.0579195 , -0.00084011, ...,  0.04882628,
         0.11374922, -0.00086206],
       ..., 
       [ 0.05012651,  0.01271051,  0.0967692 , ...,  0.04220705,
        -0.00144315, -0.08524683],
       [-0.03133665, -0.10970489, -0.14375091, ...,  0.07577346,
        -0.11832098,  0.10311176],
       [-0.12919651,  0.02734718, -0.13973653, ...,  0.0622164 ,
         0.06787138, -0.0902569 ]], dtype=float32)

In [22]:
outputs, new_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state, stacked_cell,
                                                           loop_function=None, scope='rnnlm')
new_state

(<tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_98:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_99:0' shape=(60, 128) dtype=float32>)

In [23]:
outputs[0:5]

[<tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_1:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_3:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_5:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_7:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/cell_0/basic_rnn_cell/Tanh_9:0' shape=(60, 128) dtype=float32>]

In [24]:
first_output = outputs[0]
session.run(tf.global_variables_initializer())
session.run(first_output,feed_dict={input_data:x})

array([[ 0.00397205,  0.02240689, -0.06027894, ..., -0.02570696,
        -0.07564873, -0.08115882],
       [ 0.01384854, -0.08106685,  0.08388658, ...,  0.02756775,
         0.10648959,  0.00848107],
       [-0.01470186,  0.04787109,  0.08990581, ..., -0.0360917 ,
         0.02848805,  0.13181604],
       ..., 
       [ 0.10402007, -0.00941647, -0.099663  , ...,  0.01729866,
        -0.011406  , -0.08606934],
       [ 0.0331465 ,  0.05297693,  0.12328503, ..., -0.00317745,
        -0.1081135 ,  0.08157679],
       [-0.14440858,  0.01972943, -0.03758416, ..., -0.03074711,
        -0.05441202, -0.07565662]], dtype=float32)

In [25]:
output = tf.reshape(tf.concat( outputs,1), [-1, rnn_size])
output

<tf.Tensor 'Reshape:0' shape=(3000, 128) dtype=float32>

In [26]:
logits = tf.matmul(output, softmax_w) + softmax_b
logits

<tf.Tensor 'add:0' shape=(3000, 65) dtype=float32>

In [27]:
probs = tf.nn.softmax(logits)
probs

<tf.Tensor 'Softmax:0' shape=(3000, 65) dtype=float32>

In [28]:
session.run(tf.global_variables_initializer())
session.run(probs,feed_dict={input_data:x})

array([[ 0.01104834,  0.01193885,  0.0172088 , ...,  0.01370785,
         0.01367028,  0.0173541 ],
       [ 0.01125577,  0.01214547,  0.01707018, ...,  0.01683796,
         0.01467508,  0.01461854],
       [ 0.01219858,  0.01234789,  0.01418491, ...,  0.01745815,
         0.01305982,  0.01769576],
       ..., 
       [ 0.01009449,  0.01012842,  0.01277855, ...,  0.0175134 ,
         0.01551311,  0.01781775],
       [ 0.01221049,  0.0105874 ,  0.01597082, ...,  0.01661074,
         0.01636837,  0.01460471],
       [ 0.00936382,  0.01168642,  0.01413052, ...,  0.015823  ,
         0.01304499,  0.01283296]], dtype=float32)

In [29]:
grad_clip =5.
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'rnnlm/softmax_w:0' shape=(128, 65) dtype=float32_ref>,
 <tf.Variable 'rnnlm/softmax_b:0' shape=(65,) dtype=float32_ref>,
 <tf.Variable 'rnnlm/embedding:0' shape=(65, 128) dtype=float32_ref>,
 <tf.Variable 'rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/kernel:0' shape=(256, 128) dtype=float32_ref>,
 <tf.Variable 'rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/bias:0' shape=(128,) dtype=float32_ref>]

In [30]:
class LSTMModel():
    def __init__(self,sample=False):
        rnn_size = 128 # size of RNN hidden state vector
        batch_size = 60 # minibatch size, i.e. size of dataset in each epoch
        seq_length = 50 # RNN sequence length
        num_layers = 2 # number of layers in the RNN
        vocab_size = 65
        grad_clip = 5.
        if sample:
            print(">> sample mode:")
            batch_size = 1
            seq_length = 1
        # The core of the model consists of an LSTM cell that processes one char at a time and computes probabilities of the possible continuations of the char. 
        basic_cell = tf.contrib.rnn.BasicRNNCell(rnn_size)
        # model.cell.state_size is (128, 128)
        self.stacked_cell = tf.contrib.rnn.MultiRNNCell([basic_cell] * num_layers)

        self.input_data = tf.placeholder(tf.int32, [batch_size, seq_length], name="input_data")
        self.targets = tf.placeholder(tf.int32, [batch_size, seq_length], name="targets")
        # Initial state of the LSTM memory.
        # The memory state of the network is initialized with a vector of zeros and gets updated after reading each char. 
        self.initial_state = stacked_cell.zero_state(batch_size, tf.float32) #why batch_size

        with tf.variable_scope('rnnlm_class1'):
            softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size]) #128x65
            softmax_b = tf.get_variable("softmax_b", [vocab_size]) # 1x65
            with tf.device("/gpu:0"):
                embedding = tf.get_variable("embedding", [vocab_size, rnn_size])  #65x128
                inputs = tf.split(tf.nn.embedding_lookup(embedding, self.input_data), seq_length, 1)
                inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
                #inputs = tf.split(em, seq_length, 1)
                
                


        # The value of state is updated after processing each batch of chars.
        outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, self.initial_state, self.stacked_cell, loop_function=None, scope='rnnlm_class1')
        output = tf.reshape(tf.concat(outputs,1), [-1, rnn_size])
        self.logits = tf.matmul(output, softmax_w) + softmax_b
        self.probs = tf.nn.softmax(self.logits)
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([self.logits],
                [tf.reshape(self.targets, [-1])],
                [tf.ones([batch_size * seq_length])],
                vocab_size)
        self.cost = tf.reduce_sum(loss) / batch_size / seq_length
        self.final_state = last_state
        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars),grad_clip)
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))
    
    
    def sample(self, sess, chars, vocab, num=200, prime='The ', sampling_type=1):
        state = sess.run(self.stacked_cell.zero_state(1, tf.float32))
        #print state
        for char in prime[:-1]:
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state:state}
            [state] = sess.run([self.final_state], feed)

        def weighted_pick(weights):
            t = np.cumsum(weights)
            s = np.sum(weights)
            return(int(np.searchsorted(t, np.random.rand(1)*s)))

        ret = prime
        char = prime[-1]
        for n in range(num):
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state:state}
            [probs, state] = sess.run([self.probs, self.final_state], feed)
            p = probs[0]

            if sampling_type == 0:
                sample = np.argmax(p)
            elif sampling_type == 2:
                if char == ' ':
                    sample = weighted_pick(p)
                else:
                    sample = np.argmax(p)
            else: # sampling_type == 1 default:
                sample = weighted_pick(p)

            pred = chars[sample]
            ret += pred
            char = pred
        return ret

In [31]:
with tf.variable_scope("rnn"):
    model = LSTMModel()

In [32]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(num_epochs): # num_epochs is 5 for test, but should be higher
        sess.run(tf.assign(model.lr, learning_rate * (decay_rate ** e)))
        data_loader.reset_batch_pointer()
        state = sess.run(model.initial_state) # (2x[60x128])
        for b in range(data_loader.num_batches): #for each batch
            start = time.time()
            x, y = data_loader.next_batch()
            feed = {model.input_data: x, model.targets: y, model.initial_state:state}
            train_loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)
            end = time.time()
        print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}" \
                .format(e * data_loader.num_batches + b, num_epochs * data_loader.num_batches, e, train_loss, end - start))
        with tf.variable_scope("rnn", reuse=True):
            sample_model = LSTMModel(sample=True)
            print sample_model.sample(sess, data_loader.chars , data_loader.vocab, num=50, prime='The ', sampling_type=1)
            print '----------------------------------'

370/46375 (epoch 0), train_loss = 2.001, time/batch = 0.007
>> sample mode:
The I'df his him

LoEd: dy hlo. and?
Ond:
eseriing ave
----------------------------------
741/46375 (epoch 1), train_loss = 1.800, time/batch = 0.008
>> sample mode:
The spay love. 'Twarce:
Maktad'd sleosed centy debes a
----------------------------------
1112/46375 (epoch 2), train_loss = 1.712, time/batch = 0.007
>> sample mode:
The druch I will
mior your hease, theur.

CAMILLO:
Wat
----------------------------------
1483/46375 (epoch 3), train_loss = 1.662, time/batch = 0.007
>> sample mode:
The Hespeeps churwer her doye?

GLOUCESTER:
Nancemsed:
----------------------------------
1854/46375 (epoch 4), train_loss = 1.629, time/batch = 0.009
>> sample mode:
The huth we should be there you for may here much it
b
----------------------------------
2225/46375 (epoch 5), train_loss = 1.604, time/batch = 0.011
>> sample mode:
The fire.

LEONTES:
I wloud paris.

Edrpenfatchtay ban
----------------------------------


18549/46375 (epoch 49), train_loss = 1.433, time/batch = 0.007
>> sample mode:
The night! Kate creeting wisdon?
Rumement set how hear
----------------------------------
18920/46375 (epoch 50), train_loss = 1.432, time/batch = 0.007
>> sample mode:
The cell, she stir and with a ease unto mercy, the
me 
----------------------------------
19291/46375 (epoch 51), train_loss = 1.432, time/batch = 0.007
>> sample mode:
The did
Derpt my grock:
After I corn'd all
thy fight, 
----------------------------------
19662/46375 (epoch 52), train_loss = 1.431, time/batch = 0.008
>> sample mode:
The fell you;
I' that what might to the live upon hath
----------------------------------
20033/46375 (epoch 53), train_loss = 1.430, time/batch = 0.010
>> sample mode:
The did Ksee have nours. Take their and me: the other 
----------------------------------
20404/46375 (epoch 54), train_loss = 1.430, time/batch = 0.009
>> sample mode:
The proportion to well-bens!
What has court
Of uncle, 
---------------------

36728/46375 (epoch 98), train_loss = 1.415, time/batch = 0.007
>> sample mode:
The judgry, oment for my side him was did by Glayted o
----------------------------------
37099/46375 (epoch 99), train_loss = 1.415, time/batch = 0.007
>> sample mode:
The secutions will from her,
Will have bear my feeging
----------------------------------
37470/46375 (epoch 100), train_loss = 1.414, time/batch = 0.007
>> sample mode:
The obdond good slown his fender reported by war befki
----------------------------------
37841/46375 (epoch 101), train_loss = 1.414, time/batch = 0.007
>> sample mode:
The woull nease' of not made
By at, in my mouths,
To c
----------------------------------
38212/46375 (epoch 102), train_loss = 1.414, time/batch = 0.010
>> sample mode:
The old eveny danger'd of the treatic God!
They sport 
----------------------------------
38583/46375 (epoch 103), train_loss = 1.414, time/batch = 0.010
>> sample mode:
The wounds me night;
My sadient himself, what tell as 
-----------------